<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/results/AD4_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu121
cuda:  True


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet
! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet

In [3]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [4]:
from scripts.func_utils import make_path, convert_with_qed_sa, get_min, \
                                plot_loss, plot_performance

from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *
import yaml
import pandas as pd
from scripts.get_vocab import *

from tdc.single_pred import ADME
from tdc.single_pred import Tox
from scripts.CONSTANT import *
from scripts.yaml_utils import *
from scripts.TRAIN import *
print('VOCAB TYPE:', VOCAB_TYPE)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
VOCAB TYPE: smiles


# 3 absorption tasks and 1 distribution task BBB, trained separately or together, results


    ['BBB_Martins',
    'PAMPA_NCATS',
    'Lipophilicity_AstraZeneca',
    'Solubility_AqSolDB']

In [5]:
names_AD

['BBB_Martins',
 'PAMPA_NCATS',
 'Lipophilicity_AstraZeneca',
 'Solubility_AqSolDB']

## Single task

In [6]:
print('single task evaluation on different models\n')
best_model_dict = {}
for name in names_AD:
    model_dict = {}
    print('Evaluating ', name)
    folder_name = f'ST/{name}/'
    for model_type in ['MLP', 'AttentiveFP', 'GIN', 'RNN']:
        print('\n', '='*30, model_type, '='*30)
        perfs = []
        for i in range(3):
            file_name = folder_name + model_type + '_ST_' + str(i) + '.yml'
            p = yml_report(file_name, recalculate=False)
            perfs.append(p)
        best_idx = eval_perf_list(perfs, name)
        model_dict[model_type] = best_idx
    best_model_dict[name] = model_dict
    print('\n\n')

single task evaluation on different models

Evaluating  BBB_Martins

 ============================== MLP ==============================
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.842$\pm$0.012  &0.874$\pm$0.003  &0.946$\pm$0.001  
 idx 1: &0.860            &0.877            &0.948            


 ============================== AttentiveFP ==============================
repeated num # 3 idx 2 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.851$\pm$0.004  &0.853$\pm$0.017  &0.925$\pm$0.011  
 idx 2: &0.852            &0.853            &0.929            


 ============================== GIN ==============================
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.856$\pm$0.003  &0.909$\pm$

## Multi-task without uncertainty weight

In [7]:
print('multi task evaluation on different models, without uncertainty weight\n')
best_model_dict = {}
for name in names_AD:
    model_dict = {}
    print('Evaluating ', name)
    folder_name = f'AD_mix/uw_no/'
    for model_type in ['MLP', 'AttentiveFP', 'GIN', 'RNN']:
        print('\n', '='*30, model_type, '='*30)
        perfs = []
        for i in range(3):
            file_name = folder_name + model_type + '_MT_' + str(i) + '.yml'
            p = yml_report(file_name, recalculate=False)
            perfs.append(p)
        best_idx = eval_perf_list(perfs, name)
        model_dict[model_type] = best_idx
    best_model_dict[name] = model_dict
    print('\n\n')

multi task evaluation on different models, without uncertainty weight

Evaluating  BBB_Martins

 ============================== MLP ==============================
repeated num # 3 idx 0 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.841$\pm$0.003  &0.870$\pm$0.002  &0.945$\pm$0.002  
 idx 0: &0.837            &0.868            &0.943            


 ============================== AttentiveFP ==============================
repeated num # 3 idx 0 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.851$\pm$0.007  &0.881$\pm$0.001  &0.945$\pm$0.002  
 idx 0: &0.860            &0.883            &0.947            


 ============================== GIN ==============================
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0

## Multi-task with uncertainty weight

In [8]:
print('multi task evaluation on different models, with uncertainty weight\n')
best_model_dict = {}
for name in names_AD:
    model_dict = {}
    print('Evaluating ', name)
    folder_name = f'AD_mix/uw/'
    for model_type in ['MLP', 'AttentiveFP', 'GIN', 'RNN']:
        print('\n', '='*30, model_type, '='*30)
        perfs = []
        for i in range(3):
            file_name = folder_name + model_type + '_MT_' + str(i) + '.yml'
            p = yml_report(file_name, recalculate=False)
            perfs.append(p)
        best_idx = eval_perf_list(perfs, name)
        model_dict[model_type] = best_idx
    best_model_dict[name] = model_dict
    print('\n\n')

multi task evaluation on different models, with uncertainty weight

Evaluating  BBB_Martins

 ============================== MLP ==============================
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.850$\pm$0.006  &0.863$\pm$0.002  &0.940$\pm$0.002  
 idx 1: &0.850            &0.865            &0.942            


 ============================== AttentiveFP ==============================
repeated num # 3 idx 1 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.851$\pm$0.005  &0.875$\pm$0.011  &0.946$\pm$0.008  
 idx 1: &0.850            &0.880            &0.950            


 ============================== GIN ==============================
repeated num # 3 idx 2 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.86

## MUE

In [9]:
folder_name = 'AD_mix/uw/'
print('MUE')
for name in names_AD:
    print('Evaluating ', name)
    perfs = []
    for i in range(3):
        file_name = folder_name + 'MUE' + '_'+ name + '_' + str(i) + '.yml'
        p = yml_report(file_name, recalculate=False)
        perfs.append(p)
    eval_perf_list(perfs, name)


MUE
Evaluating  BBB_Martins
repeated num # 3 idx 0 has the lowest loss
******************** BBB_Martins ******************** 
	|       acc      |       auc      |       ap      
	&0.862$\pm$0.002  &0.909$\pm$0.004  &0.966$\pm$0.001  
 idx 0: &0.862            &0.912            &0.967            

Evaluating  PAMPA_NCATS
repeated num # 3 idx 0 has the lowest loss
******************** PAMPA_NCATS ******************** 
	|       acc      |       auc      |       ap      
	&0.858$\pm$0.010  &0.821$\pm$0.001  &0.958$\pm$0.000  
 idx 0: &0.870            &0.819            &0.958            

Evaluating  Lipophilicity_AstraZeneca
repeated num # 3 idx 1 has the lowest loss
******************** Lipophilicity_AstraZeneca ******************** 
	|       mae      |       rmse      |       r2      
	&0.502$\pm$0.014  &0.658$\pm$0.018  &0.708$\pm$0.016  
 idx 1: &0.482            &0.633            &0.730            

Evaluating  Solubility_AqSolDB
repeated num # 3 idx 2 has the lowest loss
***********